In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import winsound
import ipywidgets as widgets
import IPython.display as ipd
from scipy.signal import butter, filtfilt
from IPython.display import display, Audio
from scipy.signal import chirp
from scipy.io.wavfile import write

In [2]:
audio_file = r'<Your Audio File Here>'
signal, sample_rate = librosa.load(audio_file, sr=None)
interval = 0.20 #Set Interval right here
chunk_size = int(sample_rate*interval)

In [3]:
#Beep Generation Function
def play_beep(sample_rate, dominant_frequencies):
    sound = np.array([])
    t = np.linspace(0, interval, int(sample_rate*interval), endpoint=False)
    phase = 0

    for freq in dominant_frequencies:
            beep = 0.15 * np.sin(2 * np.pi * freq * t + phase)
            phase = (phase + 2 * np.pi * freq * interval) % (2 * np.pi)
            sound = np.concatenate((sound, beep))

    return sound

#Bandpass filter function for frequency extraction
def bandpass_filter(data, lowcut, highcut, sample_rate, order=4):
    nyquist = 0.5 * sample_rate
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype = 'band', analog = False)
    return filtfilt(b, a, data)

In [4]:
#Frequency extraction using FFT
dominant_freqs = []
filtered_signal = bandpass_filter(signal, 200,  4000, sample_rate=sample_rate)
magnitude_threshold = 50
for i in range(0, len(signal), chunk_size):
    chunk = filtered_signal[i:i + chunk_size]
    
    # Pad the chunk if it's less than 1 second (for the last segment)
    if len(chunk) < chunk_size:
        chunk = np.pad(chunk, (0, chunk_size - len(chunk)))
    
    # Perform FFT
    fft_result = np.fft.fft(chunk)
    fft_magnitude = np.abs(fft_result)  # Get magnitude of FFT
    frequencies = np.fft.fftfreq(len(fft_result), 1 / sample_rate)  # Get frequency bins

    # Keep only positive frequencies
    freqs = frequencies[:len(frequencies) // 2]
    magnitudes = fft_magnitude[:len(frequencies) // 2]

    # Find dominant frequency
    dominant_frequency = freqs[np.argmax(magnitudes)]
    max_index = np.argmax(magnitudes)
    max_magnitude = magnitudes[max_index]
   
    #Lambda Function(could be useful for transforming frequency to music note)
    #note_frequencies = { #128, 132, 164, 196, 292, 328, 388, 392, 440, 492, 496, 524, 588, 592, 660, 744, 788
    #    "0": 0, "1": 128, "2": 132, "3": 164, "4": 196, "5": 292, "6": 328, "7": 388, "8": 392, "9": 440, 
    #    "10": 492, "11": 496, "12": 524, "13": 588, "14": 592, "15": 660, "16": 744, "17": 788
    #    "0": 0.00, "C0": 16.35, "C#0": 17.32, "D0": 18.35, "D#0": 19.45, "E0": 20.60, 
    #    "F0": 21.83, "F#0": 23.12, "G0": 24.50, "G#0": 25.96, "A0": 27.50, "A#0": 29.14, "B0": 30.87,
    #    "C1": 32.70, "C#1": 34.65, "D1": 36.71, "D#1": 38.89, "E1": 41.20, 
    #    "F1": 43.65, "F#1": 46.25, "G1": 49.00, "G#1": 51.91, "A1": 55.00, "A#1": 58.27, "B1": 61.74,
    #    "C2": 65.41, "C#2": 69.30, "D2": 73.42, "D#2": 77.78, "E2": 82.41, 
    #    "F2": 87.31, "F#2": 92.50, "G2": 98.00, "G#2": 103.83, "A2": 110.00, "A#2": 116.54, "B2": 123.47,
    #    "C3": 130.81, "C#3": 138.59, "D3": 146.83, "D#3": 155.56, "E3": 164.81, 
    #    "F3": 174.61, "F#3": 185.00, "G3": 196.00, "G#3": 207.65, "A3": 220.00, "A#3": 233.08, "B3": 246.94,
    #    "C4": 261.63, "C#4": 277.18, "D4": 293.66, "D#4": 311.13, "E4": 329.63, 
    #    "F4": 349.23, "F#4": 369.99, "G4": 392.00, "G#4": 415.30, "A4": 440.00, "A#4": 466.16, "B4": 493.88,
    #    "C5": 523.25, "C#5": 554.37, "D5": 587.33, "D#5": 622.25, "E5": 659.25, 
    #    "F5": 698.46, "F#5": 739.99, "G5": 783.99, "G#5": 830.61, "A5": 880.00, "A#5": 932.33, "B5": 987.77,
    #    "C6": 1046.50, "C#6": 1108.73, "D6": 1174.66, "D#6": 1244.51, "E6": 1318.51, 
    #    "F6": 1396.91, "F#6": 1479.98, "G6": 1567.98, "G#6": 1661.22, "A6": 1760.00, "A#6": 1864.66, "B6": 1975.53
    #}
    #closest_note = min(note_frequencies, key=lambda note: abs(dominant_frequency - note_frequencies[note]))
    #dominant_frequency = (note_frequencies[closest_note])
    
    
    if max_magnitude > magnitude_threshold:  
        dominant_freqs.append(dominant_frequency)
        #dominant_magnitudes.append(max_magnitude)
    else:
        dominant_freqs.append(0)
        #dominant_magnitudes.append(0)

    # Normalize magnitudes to [0, 1] for amplitude control
    #max_mag = max(dominant_magnitudes) if max(dominant_magnitudes) > 0 else 1
    #normalized_amplitudes = [mag / max_mag for mag in dominant_magnitudes]
    """
    # Plot the spectrum
    plt.figure(figsize=(10, 4))
    plt.plot(freqs, magnitudes)
    plt.title(f"Frequency Spectrum for Second {i // chunk_size + 1}")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude")
    plt.xlim(0,10000)
    plt.grid()
    plt.annotate(f"{dominant_frequency:.2f} Hz",
                 xy=(dominant_frequency, np.max(magnitudes)),
                 xytext=(dominant_frequency, np.max(magnitudes) * 1.1),
                 arrowprops=dict(arrowstyle="->", color="red"),
                 color="red")
    plt.show()
    print(f"Second {i // chunk_size + 1}: Dominant Frequency = {dominant_frequency:.2f} Hz")
    """

In [5]:
beep = play_beep(sample_rate, dominant_freqs)
output_filename = "output.wav"
write(output_filename, sample_rate, (beep * 32767).astype(np.int16))

In [ ]:
#For Testing Purpose
#min_freq = min(dominant_freqs)
#max_freq = max(dominant_freqs)
#print(f"Minimum dominant frequency: {min_freq:.2f} Hz")
#print(f"Maximum dominant frequency: {max_freq:.2f} Hz")

In [ ]:
#For Testing Purpose
#cleaned_data = [float(x) for x in dominant_freqs]
#print(cleaned_data)